In [ ]:
import tensorflow as tf
import pandas as pd
from PIL import Image
from io import BytesIO
import io
import matplotlib.pyplot as plt
import os

import glob
import xml.etree.ElementTree as ET

In [ ]:
def class_text_to_int(row_label):
    if row_label == 'batu':
        return 1
    elif row_label == 'butir gabah':
        return 2
    elif row_label == 'butir kapur':
        return 3
    elif row_label == 'butir kepala':
        return 4
    elif row_label == 'butir menir':
        return 5
    elif row_label == 'butir merah':
        return 6
    elif row_label == 'butir patah':
        return 7
    elif row_label == 'butir rusak':
        return 8
    elif row_label == 'kutu':
        return 9
    elif row_label == 'sekam':
        return 10
    else:
        return 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Dataset/dataset.zip

Archive:  /content/drive/MyDrive/Dataset/dataset.zip
   creating: dataset/
   creating: dataset/test/
  inflating: dataset/test/IMG_0680.jpg  
  inflating: dataset/test/IMG_0680.xml  
  inflating: dataset/test/IMG_0681.jpg  
  inflating: dataset/test/IMG_0681.xml  
  inflating: dataset/test/IMG_0682.jpg  
  inflating: dataset/test/IMG_0682.xml  
  inflating: dataset/test/IMG_0683.jpg  
  inflating: dataset/test/IMG_0683.xml  
  inflating: dataset/test/IMG_0684.jpg  
  inflating: dataset/test/IMG_0684.xml  
  inflating: dataset/test/IMG_0685.jpg  
  inflating: dataset/test/IMG_0685.xml  
  inflating: dataset/test/IMG_0687.jpg  
  inflating: dataset/test/IMG_0687.xml  
  inflating: dataset/test/IMG_0688.jpg  
  inflating: dataset/test/IMG_0688.xml  
  inflating: dataset/test/IMG_0689.jpg  
  inflating: dataset/test/IMG_0689.xml  
  inflating: dataset/test/IMG_0690.jpg  
  inflating: dataset/test/IMG_0690.xml  
  inflating: dataset/test/IMG_0691.jpg  
  inflating: dataset/test/IMG_0691.xm

In [ ]:
def xml_to_csv(xml_folder, csv_output):
    xml_list = []
    for xml_file in glob.glob(os.path.join(xml_folder, '*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text))
            xml_list.append(value)

    column_names = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_names)
    xml_df.to_csv(csv_output, index=None)
    print(f'CSV file saved to {csv_output}')

# Contoh penggunaan:
xml_folder = '/content/dataset/train'
csv_output = 'train.csv'
xml_to_csv(xml_folder, csv_output)

CSV file saved to train.csv


In [ ]:
def xml_to_csv_test(xml_folder, csv_output):
    xml_list = []
    for xml_file in glob.glob(os.path.join(xml_folder, '*.xml')):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text))
            xml_list.append(value)

    column_names = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_names)
    xml_df.to_csv(csv_output, index=None)
    print(f'CSV file saved to {csv_output}')

# Contoh penggunaan:
xml_folder = '/content/dataset/test'
csv_output = 'test.csv'
xml_to_csv(xml_folder, csv_output)

CSV file saved to test.csv


In [ ]:
import os
import cv2
import pandas as pd

def resize_image(image_path, output_path, target_size=(640, 640)):
    image = cv2.imread(image_path)

    # Faktor Resize
    resize_factor = min(target_size[0] / image.shape[1], target_size[1] / image.shape[0])

    # Resize gambar
    resized_image = cv2.resize(image, None, fx=resize_factor, fy=resize_factor)

    # Simpan gambar yang sudah diresize
    cv2.imwrite(output_path, resized_image)

    return resized_image, resize_factor

def update_csv(csv_path, resize_factor):
    df = pd.read_csv(csv_path)

    for index, row in df.iterrows():
        # Hitung ulang koordinat bounding box
        df.at[index, 'xmin'] = int(row['xmin'] * resize_factor)
        df.at[index, 'ymin'] = int(row['ymin'] * resize_factor)
        df.at[index, 'xmax'] = int(row['xmax'] * resize_factor)
        df.at[index, 'ymax'] = int(row['ymax'] * resize_factor)
        df.at[index, 'width'] = int(row['width'] * resize_factor)
        df.at[index, 'height'] = int(row['height'] * resize_factor)

    # Menyimpan file CSV yang telah diperbarui sebagai file baru
    new_csv_path = csv_path.replace('.csv', '_resized_640.csv')
    df.to_csv(new_csv_path, index=False)
    return new_csv_path

def resize_images_in_folder(input_folder, output_folder, target_size=(640, 640)):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Sesuaikan dengan jenis gambar yang digunakan
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Resize gambar dan perbarui CSV
            resized_image, resize_factor = resize_image(input_path, output_path, target_size)

    # Perbarui file CSV dengan koordinat yang sesuai
    dt_input_folder = '/content'
    csv_path = os.path.join(dt_input_folder, 'train.csv')  # Sesuaikan dengan nama file CSV Anda
    new_csv_path = update_csv(csv_path, resize_factor)

    return new_csv_path

# Contoh pemanggilan fungsi
input_folder_path = '/content/dataset/train'
output_folder_path = '/content/dataset/train_resize_640'
new_csv_path = resize_images_in_folder(input_folder_path, output_folder_path)

print(f"File CSV yang diperbarui disimpan sebagai: {new_csv_path}")

File CSV yang diperbarui disimpan sebagai: /content/train_resized_640.csv


In [ ]:
import os
import cv2
import pandas as pd

def resize_image(image_path, output_path, target_size=(640, 640)):
    image = cv2.imread(image_path)

    # Faktor Resize
    resize_factor = min(target_size[0] / image.shape[1], target_size[1] / image.shape[0])

    # Resize gambar
    resized_image = cv2.resize(image, None, fx=resize_factor, fy=resize_factor)

    # Simpan gambar yang sudah diresize
    cv2.imwrite(output_path, resized_image)

    return resized_image, resize_factor

def update_csv(csv_path, resize_factor):
    df = pd.read_csv(csv_path)

    for index, row in df.iterrows():
        # Hitung ulang koordinat bounding box
        df.at[index, 'xmin'] = int(row['xmin'] * resize_factor)
        df.at[index, 'ymin'] = int(row['ymin'] * resize_factor)
        df.at[index, 'xmax'] = int(row['xmax'] * resize_factor)
        df.at[index, 'ymax'] = int(row['ymax'] * resize_factor)
        df.at[index, 'width'] = int(row['width'] * resize_factor)
        df.at[index, 'height'] = int(row['height'] * resize_factor)

    # Menyimpan file CSV yang telah diperbarui sebagai file baru
    new_csv_path = csv_path.replace('.csv', '_resized_640.csv')
    df.to_csv(new_csv_path, index=False)
    return new_csv_path

def resize_images_in_folder(input_folder, output_folder, target_size=(640, 640)):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Sesuaikan dengan jenis gambar yang digunakan
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Resize gambar dan perbarui CSV
            resized_image, resize_factor = resize_image(input_path, output_path, target_size)

    # Perbarui file CSV dengan koordinat yang sesuai
    dt_input_folder = '/content'
    csv_path = os.path.join(dt_input_folder, 'test.csv')  # Sesuaikan dengan nama file CSV Anda
    new_csv_path = update_csv(csv_path, resize_factor)

    return new_csv_path

# Contoh pemanggilan fungsi
input_folder_path = '/content/dataset/test'
output_folder_path = '/content/dataset/test_resize_640'
new_csv_path = resize_images_in_folder(input_folder_path, output_folder_path)

print(f"File CSV yang diperbarui disimpan sebagai: {new_csv_path}")

File CSV yang diperbarui disimpan sebagai: /content/test_resized_640.csv


In [ ]:
image_dir = '/content/dataset/train_resize_640'

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# Fungsi konversi tipe data untuk fitur numerik
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[int(value)]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


# Update your create_tf_example function to use the original image
def create_tf_example(row):
    # Mengonversi gambar menjadi bytes
    image_path = os.path.join(image_dir, row['filename'])

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()

    feature = {
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'jpeg'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=[row['xmin']])),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=[row['xmax']])),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=[row['ymin']])),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=[row['ymax']])),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=[class_text_to_int(row['class'])])),
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example

csv_file_path = '/content/train_resized_640.csv'

# Membaca file CSV
df = pd.read_csv(csv_file_path)

# Gantilah dengan path untuk menyimpan file TFRecord
output_tfrecord_path = '/content/training.tfrecord'

# Menulis data ke dalam file TFRecord
with tf.io.TFRecordWriter(output_tfrecord_path) as writer:
    for index, row in df.iterrows():
        tf_example = create_tf_example(row)
        writer.write(tf_example.SerializeToString())

print(f"TFRecord file created at: {output_tfrecord_path}")

TFRecord file created at: /content/training.tfrecord


In [ ]:
image_dir = '/content/dataset/test_resize_640'

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# Fungsi konversi tipe data untuk fitur numerik
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[int(value)]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


# Update your create_tf_example function to use the original image
def create_tf_example(row):
    # Mengonversi gambar menjadi bytes
    image_path = os.path.join(image_dir, row['filename'])

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()

    feature = {
        'image/encoded': _bytes_feature(encoded_image),
        'image/format': _bytes_feature(b'jpeg'),
        'image/object/bbox/xmin': _float_feature(row['xmin']),
        'image/object/bbox/xmax': _float_feature(row['xmax']),
        'image/object/bbox/ymin': _float_feature(row['ymin']),
        'image/object/bbox/ymax': _float_feature(row['ymax']),
        'image/object/class/label': _int64_feature(class_text_to_int(row['class'])),
    }

    tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
    return tf_example

csv_file_path = '/content/test_resized_640.csv'

# Membaca file CSV
df = pd.read_csv(csv_file_path)

# Gantilah dengan path untuk menyimpan file TFRecord
output_tfrecord_path = '/content/test.tfrecord'

# Menulis data ke dalam file TFRecord
with tf.io.TFRecordWriter(output_tfrecord_path) as writer:
    for index, row in df.iterrows():
        tf_example = create_tf_example(row)
        writer.write(tf_example.SerializeToString())

print(f"TFRecord file created at: {output_tfrecord_path}")

TFRecord file created at: /content/test.tfrecord
